<a href="https://colab.research.google.com/github/minthammock-development/Technical-Assignments/blob/main/RingCentral_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd
import numpy as np

In [170]:
# Custom Functions
def breakColumns(df, columns, conditionRevenue = []):
  for column in columns:
    entries = df[column].value_counts().index
    for entry in entries:
      df[entry] = [1 if x == entry else 0 for x in df[column]]
      if column in conditionRevenue:
        df[f'{entry} Revenue'] = [df['Forecasted Annual Revenue'][i] if (df[column][i] == entry and df['Stage'][i] == '7. Closed Won') else 0 for i in range(len(df))]
        df[f'{entry} Lost Revenue'] = [df['Forecasted Annual Revenue'][i] if (df[column][i] == entry and df['Stage'][i] == '0. Downgraded') else 0 for i in range(len(df))]


In [171]:
dfPartners = pd.read_excel('/content/drive/MyDrive/Reporting Working Example-S&D Details-Partner Operations Analyst.xlsx',sheet_name='Partners')
dfOpp = pd.read_excel('/content/drive/MyDrive/Reporting Working Example-S&D Details-Partner Operations Analyst.xlsx', sheet_name='Opportunities')

In [172]:
display(dfPartners.loc[dfPartners['Partner ID'].duplicated() == True])
dfPartners = dfPartners.drop(dfPartners.loc[dfPartners['Partner ID'].duplicated() == True].index)

,Partner Name,Partner ID,Channel Manager
186,Partner 536,44764,Channel Manager 6
271,Partner 543,56304,Channel Manager 6
298,Partner 490,58007,Channel Manager 4
308,Partner 599,59128,Channel Manager 7
454,Partner 556,74611,Channel Manager 6
456,Partner 533,75119,Channel Manager 5
457,Partner 557,75119,Channel Manager 6
493,Partner 617,79257,Channel Manager 7
498,Partner 618,79265,Channel Manager 7
514,Partner 566,79349,Channel Manager 6


In [173]:
# display(dfPartners.head())
# dfPartners.info()
# dfPartners['Partner ID'].value_counts()


In [174]:
# display(dfOpp.head())
# dfOpp.info()

In [175]:
df = pd.merge(dfPartners, dfOpp, on='Partner ID')
df.set_index('CreateDateTime', inplace=True)
df.sort_values('CreateDateTime', axis = 0, inplace=True)
df['Opportunity'] = [1 for x in df['Opportunity Name']]
df['Running Opportunities'] = df['Opportunity'].cumsum()
df['Closed Revenue'] = [df['Forecasted Annual Revenue'][i] if df['Stage'][i] == '7. Closed Won' else 0 for i in range(len(df))]
df['Lost Revenue'] = [df['Forecasted Annual Revenue'][i] if df['Stage'][i] == '0. Downgraded' else 0 for i in range(len(df))] 
df['Cumulative Forecasted Revenue'] = df['Forecasted Annual Revenue'].cumsum()
df['Cumulative Closed Revenue'] = df['Closed Revenue'].cumsum()
df['Cumulative Lost Revenue'] = df['Lost Revenue'].cumsum()

columnsToBreak = ['Channel Manager', 'Partner Name','Stage' ]
breakColumns(df, columnsToBreak, conditionRevenue=columnsToBreak[:2])
df.head()

,Partner Name,Partner ID,Channel Manager,Opportunity Name,Forecasted Office Users,Forecasted Annual Revenue,Forecasted Close Date,Stage,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Partner 506,Partner 506 Revenue,Partner 506 Lost Revenue,Partner 562,...,Partner 481 Revenue,Partner 481 Lost Revenue,Partner 385,Partner 385 Revenue,Partner 385 Lost Revenue,Partner 238,Partner 238 Revenue,Partner 238 Lost Revenue,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03 14:23:00,Partner 269,30691,Channel Manager 3,Opportunity 1540,2.0,671.52,2020-01-08,7. Closed Won,1,1,671.52,0.00,671.52,671.52,0.00,0,0.0,0.0,0,0.0,0.0,1,671.52,0.0,0,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,0,1,0,0,0,0,0,0
2020-01-05 12:50:00,Partner 600,60082,Channel Manager 7,Opportunity 4424,1000.0,204191.77,2021-06-30,0. Downgraded,1,2,0.00,204191.77,204863.29,671.52,204191.77,0,0.0,0.0,0,0.0,0.0,0,0.00,0.0,0,0.0,0.0,1,0.0,204191.77,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,1,0,0,0,0,0,0,0
2020-01-05 12:51:00,Partner 600,60082,Channel Manager 7,Opportunity 4425,10000.0,2041917.74,2020-11-02,0. Downgraded,1,3,0.00,2041917.74,2246781.03,671.52,2246109.51,0,0.0,0.0,0,0.0,0.0,0,0.00,0.0,0,0.0,0.0,1,0.0,2041917.74,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,1,0,0,0,0,0,0,0
2020-01-05 12:52:00,Partner 600,60082,Channel Manager 7,Opportunity 4426,1000.0,0.00,2020-10-29,0. Downgraded,1,4,0.00,0.00,2246781.03,671.52,2246109.51,0,0.0,0.0,0,0.0,0.0,0,0.00,0.0,0,0.0,0.0,1,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,1,0,0,0,0,0,0,0
2020-01-05 12:56:00,Partner 600,60082,Channel Manager 7,Opportunity 4427,800.0,163353.42,2021-07-31,0. Downgraded,1,5,0.00,163353.42,2410134.45,671.52,2409462.93,0,0.0,0.0,0,0.0,0.0,0,0.00,0.0,0,0.0,0.0,1,0.0,163353.42,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,1,0,0,0,0,0,0,0


In [176]:
px.bar(x = df['Partner Name'].value_counts()[:25].index, y = df['Partner Name'].value_counts()[:25])

In [177]:
dfMonth = df.resample('MS').sum()
dfMonth['Total Churn'] = 1 - round(dfMonth['7. Closed Won']/dfMonth['Opportunity'],2)
display((dfMonth.head()))
dfQuarter = df.resample('Q').sum()
dfQuarter['Total Churn'] = 1 - round(dfQuarter['7. Closed Won']/dfQuarter['Opportunity'],2)


,Partner ID,Forecasted Office Users,Forecasted Annual Revenue,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Partner 506,Partner 506 Revenue,Partner 506 Lost Revenue,Partner 562,Partner 562 Revenue,Partner 562 Lost Revenue,Partner 528,Partner 528 Revenue,Partner 528 Lost Revenue,...,Partner 481 Lost Revenue,Partner 385,Partner 385 Revenue,Partner 385 Lost Revenue,Partner 238,Partner 238 Revenue,Partner 238 Lost Revenue,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order,Total Churn
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,5275597,50211.0,7830957.38,94,4465,27675.43,7639730.01,4.407861e+08,1600811.92,4.380699e+08,12,3095.28,1002681.27,3,0.00,54296.15,33,13568.50,743228.42,19,6130.85,554747.08,10,0.00,3043023.80,17,4880.80,2241753.29,0,0.00,0.00,4,0.00,423225.57,0,0.0,0.0,13,4880.80,2170035.14,...,0,0,0.00,0,1,0,23994.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,72,19,1,2,0,0,0,0,0.80
2020-02-01,5910729,28899.0,5705890.49,105,15435,68431.92,4886143.20,1.059132e+09,7697025.99,9.954921e+08,26,19737.73,991900.98,9,0.00,899721.94,21,8089.42,165015.62,23,18951.64,747235.73,3,0.00,241909.88,18,0.00,1775570.13,5,21653.13,64788.92,2,0.00,47126.02,0,0.0,0.0,6,0.00,1459192.35,...,0,0,0.00,0,0,0,0.0,0,0,0.0,1,0,1728.32,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,84,14,3,3,1,0,0,0,0.87
2020-03-01,8194289,29004.0,6316572.51,134,35711,266541.29,5764485.13,2.169905e+09,29582220.52,1.995623e+09,27,57790.41,931418.46,13,4729.58,121288.78,31,32130.46,491712.64,28,53122.87,813039.78,9,81539.78,397042.94,20,30099.04,1644661.39,6,7129.15,1365321.14,5,1696.88,138450.83,0,0.0,0.0,10,6834.69,267445.48,...,0,1,3106.74,0,0,0,0.0,1,0,754.2,0,0,0.00,0,0,0.0,1,0,502.52,0,0,0.0,1,0,1885.51,0,0,0.00,0,0.0,0,84,41,6,0,3,0,0,0,0.69
2020-04-01,5622912,19368.0,3003746.36,90,34065,94495.67,2403985.07,1.928371e+09,34276393.04,1.764165e+09,25,45690.38,633458.08,11,0.00,652090.90,26,17991.63,365662.15,11,1963.95,69386.22,8,24576.62,103754.33,7,3518.89,501278.14,2,754.20,78355.25,7,4529.47,396708.94,1,0.0,137401.1,5,0.00,281320.06,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,1,0,1871.04,0,0.0,0,58,21,5,4,1,0,1,0,0.77
2020-05-01,6156547,23696.0,5817174.95,101,47874,504838.11,5144969.14,2.675433e+09,74601544.75,2.420458e+09,16,68130.37,1241486.20,13,2765.16,1327092.16,19,30800.00,171011.95,29,48663.94,2123887.62,13,53643.04,103560.64,8,222669.56,173209.17,3,78166.04,4721.40,4,58887.87,216650.78,0,0.0,0.0,5,162914.23,173209.17,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,56,37,3,5,0,0,0,0,0.63


In [178]:
round(dfMonth['7. Closed Won']/dfMonth['Opportunity'],2)*100

CreateDateTime
2020-01-01    20.0
2020-02-01    13.0
2020-03-01    31.0
2020-04-01    23.0
2020-05-01    37.0
2020-06-01    29.0
2020-07-01    25.0
2020-08-01    32.0
2020-09-01    32.0
2020-10-01    23.0
2020-11-01    15.0
2020-12-01    20.0
2021-01-01    23.0
2021-02-01    15.0
2021-03-01    16.0
2021-04-01     6.0
Freq: MS, dtype: float64

In [186]:
import plotly.express as px
import plotly.graph_objects as go

px.line(data_frame=dfQuarter, x = dfQuarter.index, y ='Closed Revenue',)
px.line(data_frame=dfMonth, x = dfMonth.index, y = 'Closed Revenue')
px.bar(data_frame = dfMonth, x = dfMonth.index, y = dfMonth['Channel Manager 6'], barmode='stack')
px.line(data_frame=dfMonth, x = dfMonth.index, y ='Forecasted Annual Revenue', color = 'Channel Manager 1')

fig = go.Figure(
  data = [
    go.Bar(
      name = label, 
      x = dfMonth.index, 
      y = dfMonth[label],
      text = dfMonth[label],
      textposition = 'inside',
      textangle = 0,

    ) for label in sorted(dfPartners['Channel Manager'].unique())
  ],

  layout = {
      'title': 'Opportunities per Month by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Month',
          'tickangle' : -90,
          'nticks' : 16
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = dfQuarter[label],
      text = dfQuarter[label],
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Opportunities',
    x = dfQuarter.index, 
    y = dfQuarter['Opportunity'], 
    text = dfQuarter['Opportunity'],
    mode="lines+text+markers",
    # textangle = 0,
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Opportunities per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          # 'tickmode' : 'array',
          # 'tickvals' : ['05/01/2020','08/01/2020','11/01/2020','02/01/2021','05/01/2021','08/01/2020'],
          # 'ticktext' : ['Q2','Q3', 'Q4', 'Q1', 'Q2', 'Q3' ],
          'nticks' : 6,
          # 'ticklabelposition': 'outside left',
          
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)

# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Revenue'],0),
      text = round(dfQuarter[f'{label} Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Closed Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Closed Revenue'],2), 
    text = round(dfQuarter['Closed Revenue'],0),
    mode="lines+text+markers",
    # textangle = 0,
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Closed Revenue per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          # 'tickmode' : 'array',
          # 'tickvals' : ['05/01/2020','08/01/2020','11/01/2020','02/01/2021','05/01/2021','08/01/2020'],
          # 'ticktext' : ['Q2','Q3', 'Q4', 'Q1', 'Q2', 'Q3' ],
          'nticks' : 6,
          # 'ticklabelposition': 'outside left',
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)

# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Lost Revenue'],0),
      text = round(dfQuarter[f'{label} Lost Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Lost Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Lost Revenue'],0), 
    text = round(dfQuarter['Lost Revenue'],0),
    mode="lines+text+markers",
    # textangle = 0,
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Lost Revenue per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          # 'tickmode' : 'array',
          # 'tickvals' : ['05/01/2020','08/01/2020','11/01/2020','02/01/2021','05/01/2021','08/01/2020'],
          # 'ticktext' : ['Q2','Q3', 'Q4', 'Q1', 'Q2', 'Q3' ],
          'nticks' : 6,
          # 'ticklabelposition': 'outside left',
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)

# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

In [188]:
dfQuarter

,Partner ID,Forecasted Office Users,Forecasted Annual Revenue,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Partner 506,Partner 506 Revenue,Partner 506 Lost Revenue,Partner 562,Partner 562 Revenue,Partner 562 Lost Revenue,Partner 528,Partner 528 Revenue,Partner 528 Lost Revenue,...,Partner 481 Lost Revenue,Partner 385,Partner 385 Revenue,Partner 385 Lost Revenue,Partner 238,Partner 238 Revenue,Partner 238 Lost Revenue,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order,Total Churn
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-31,19380615,108114.0,19853420.38,333,55611,362648.64,18290358.34,3.669823e+09,3.888006e+07,3.429185e+09,65,80623.42,2926000.71,25,4729.58,1075306.87,85,53788.38,1399956.68,70,78205.36,2115022.59,22,81539.78,3681976.62,55,34979.84,5661984.81,11,28782.28,1430110.06,11,1696.88,608802.42,0,0.00,0.00,29,11715.49,3896672.97,...,0,1,3106.74,0,1,0,23994.0,1,0,754.2,1,0,1728.32,0,0,0.00,1,0,502.52,0,0,0.00,1,0,1885.51,0,0,0.00,0,0.00,0,240,74,10,5,4,0,0,0,0.78
2020-06-30,19843906,213395.0,17695431.84,312,152724,1551728.27,14776226.22,8.578067e+09,3.007277e+08,7.701094e+09,58,447537.70,2099678.55,41,157594.78,4442354.71,77,100244.12,2111174.25,68,88006.55,3761113.37,30,116106.88,215474.70,27,458544.70,1923230.02,11,183693.54,223200.62,17,199975.74,645714.92,2,0.00,274802.20,14,383329.90,1578532.93,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,1,0,1871.04,1,1308.63,0,185,93,13,16,4,0,1,0,0.70
2020-09-30,33407748,143046.0,23437362.58,476,420546,1666834.88,14783937.79,2.293884e+10,1.256357e+09,1.927777e+10,103,764723.91,5160318.38,88,208292.43,3838714.12,80,56734.63,303820.48,64,86310.77,1883646.44,66,216823.59,964106.35,42,165307.00,2227743.14,33,168642.55,405588.88,34,325484.20,1550278.32,28,146091.89,1762160.45,10,67333.61,630238.40,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0.00,0,263,143,22,27,16,4,0,1,0.70
2020-12-31,33841193,266915.0,15105084.77,464,628024,1665397.17,7109789.03,3.145558e+10,2.017381e+09,2.392471e+10,84,409906.26,1855892.66,115,490153.65,2326314.63,47,32762.84,200552.47,57,267889.90,301969.21,74,336005.28,958756.03,47,97548.49,933576.38,40,31130.75,532727.65,30,258617.77,942495.94,39,220513.86,87384.90,12,17287.14,376703.75,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,1,0,1296.24,0,0,0.00,1,0,112335.73,0,0,0.00,0,0,0.00,0,0.00,0,219,88,61,50,35,7,4,0,0.81
2021-03-31,31941096,140322.0,19495934.71,420,754110,771626.93,2279686.58,3.622175e+10,2.371808e+09,2.377564e+10,58,39284.84,511001.00,91,225903.82,697459.25,42,35578.18,56292.86,51,135393.86,261123.56,61,143454.79,308853.70,55,137233.70,266693.30,62,54777.74,178262.91,14,6602.04,96774.33,12,2502.65,103059.92,12,87154.05,345.66,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0.00,0,95,77,9

In [180]:
fig2 = go.Figure(data=[
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Closed Revenue']),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Channel Manager 7']),
    go.Bar(x = dfQuarter.index, y = dfQuarter.diff(1)['Closed Revenue']),
])
fig2.show()
fig4 = go.Figure(data=[
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Total Churn']),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Channel Manager 7']),
    go.Bar(x = dfQuarter.index, y = dfQuarter.diff(1)['Total Churn']),
])
fig4.show()
fig5 = go.Figure(
  data = [
    go.Scatter(
      name = label, 
      x = dfQuarter.index, 
      y = dfQuarter[f'{label} Revenue'],
      text = round(dfQuarter[f'{label} Revenue'],0),
    ) for label in sorted(dfPartners['Channel Manager'].unique())
  ] + \
  [go.Bar(name = label, x = dfQuarter.index, y = dfQuarter.diff(1)[f'{label} Revenue']) for label in sorted(dfPartners['Channel Manager'].unique())],

  layout = {
      'title': 'Opportunities per Month by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Month',
          'tickangle' : -90,
          'nticks' : 16
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)
fig5.show()

In [181]:
fig2 = go.Figure(data=[
    go.Scatter( x = df.index, y = df['Cumulative Closed Revenue']),
    go.Scatter(x = df.index, y = df['Cumulative Lost Revenue']),
])
fig2.show()

In [182]:
fig2 = go.Figure(data=[
    go.Scatter(name = "Closed Revenue", x = dfMonth.index, y = dfMonth['Closed Revenue']),
    go.Bar(name = 'Opportunities Per Month',x = dfMonth.index, y = dfMonth['Opportunity']*10000),
    go.Scatter(name = 'Lost Revenue',x = dfMonth.index, y = dfMonth['Lost Revenue']),
])
fig2.show()

In [183]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfQuarter.index, y = dfQuarter['Closed Revenue']),
    go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Lost Revenue']),
])
fig2.show()

In [184]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfMonth[:'03-01-2021'].index, y = dfMonth['Closed Revenue'].diff(1)/dfMonth['Closed Revenue']),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    # go.Scatter(x = dfQuarter.index, y = dfQuarter['Lost Revenue']),
])
fig2.show()

In [185]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfQuarter[:'03-31-2021'].index, y = dfQuarter['Closed Revenue'].diff(1)/dfQuarter['Closed Revenue']),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    # go.Scatter(x = dfQuarter.index, y = dfQuarter['Lost Revenue']),
])
fig2.show()

In [157]:
df

,Partner Name,Partner ID,Channel Manager,Opportunity Name,Forecasted Office Users,Forecasted Annual Revenue,Forecasted Close Date,Stage,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 2,Channel Manager 2 Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order,Partner 506,Partner 506 Revenue,Partner 562,...,Partner 619,Partner 619 Revenue,Partner 491,Partner 491 Revenue,Partner 432,Partner 432 Revenue,Partner 437,Partner 437 Revenue,Partner 342,Partner 342 Revenue,Partner 478,Partner 478 Revenue,Partner 387,Partner 387 Revenue,Partner 480,Partner 480 Revenue,Partner 611,Partner 611 Revenue,Partner 481,Partner 481 Revenue,Partner 385,Partner 385 Revenue,Partner 238,Partner 238 Revenue,Partner 502,Partner 502 Revenue,Partner 402,Partner 402 Revenue,Partner 421,Partner 421 Revenue,Partner 410,Partner 410 Revenue,Partner 458,Partner 458 Revenue,Partner 84,Partner 84 Revenue,Partner 298,Partner 298 Revenue,Partner 219,Partner 219 Revenue
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03 14:23:00,Partner 269,30691,Channel Manager 3,Opportunity 1540,2.0,671.52,2020-01-08,7. Closed Won,1,1,671.52,0.00,6.715200e+02,671.52,0.00,0,0.0,0,0.0,1,671.52,0,0.0,0,0.0,0,0.0,0,0.0,0,1,0,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-05 12:50:00,Partner 600,60082,Channel Manager 7,Opportunity 4424,1000.0,204191.77,2021-06-30,0. Downgraded,1,2,0.00,204191.77,2.048633e+05,671.52,204191.77,0,0.0,0,0.0,0,0.00,0,0.0,1,0.0,0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-05 12:51:00,Partner 600,60082,Channel Manager 7,Opportunity 4425,10000.0,2041917.74,2020-11-02,0. Downgraded,1,3,0.00,2041917.74,2.246781e+06,671.52,2246109.51,0,0.0,0,0.0,0,0.00,0,0.0,1,0.0,0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-05 12:52:00,Partner 600,60082,Channel Manager 7,Opportunity 4426,1000.0,0.00,2020-10-29,0. Downgraded,1,4,0.00,0.00,2.246781e+06,671.52,2246109.51,0,0.0,0,0.0,0,0.00,0,0.0,1,0.0,0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-05 12:56:00,Partner 600,60082,Channel Manager 7,Opportunity 4427,800.0,163353.42,2021-07-31,0. Downgraded,1,5,0.00,163353.42,2.410134e+06,671.52,2409462.93,0,0.0,0,0.0,0,0.00,0,0.0,1,0.0,0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 03:45:00,Partner 129,88872,Channel Manager 1,Opportunity 330,18.0,0.00,2021-06-30,1. Qualify,1,2087,0.00,0.00,1.000873e+08,6028463.80,57337368.67,0,0.0,0,0.0,0,0.00,1,0.0,0,0.0,0,0.0,0,0.0,0,0,1,0,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2021-04-21 04:21:00,Partner 554,71814,Channel Manager 6,Opportunity 806,1850.0,435915.64,2021-09-30,2. Problem,1,2088,0.00,0.00,1.005232e+08,6028463.80,57337368.67,0,0.0,1,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,1,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0.0,0,0,0,0.0,

In [158]:
top25 = list(df['Partner Name'].value_counts()[:25].index)
dfQuarter['Partner 506']

KeyError: ignored